<h1>Reccomendation System</h1>

<h2>Importing Libraries</h2>

In [1]:
import numpy as np
import random as rn
import pandas as pd
import seaborn as sns
import re
import math

<h2>Importing Database</h2>

In [2]:
apps = pd.read_csv(r"googleplaystore.csv")

<h1>Data Cleanup</h1>

In [3]:
display(apps)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10836,Sya9a Maroc - FR,FAMILY,4.5,38,53M,"5,000+",Free,0,Everyone,Education,"July 25, 2017",1.48,4.1 and up
10837,Fr. Mike Schmitz Audio Teachings,FAMILY,5.0,4,3.6M,100+,Free,0,Everyone,Education,"July 6, 2018",1.0,4.1 and up
10838,Parkinson Exercices FR,MEDICAL,NaN,3,9.5M,"1,000+",Free,0,Everyone,Medical,"January 20, 2017",1.0,2.2 and up
10839,The SCP Foundation DB fr nn5n,BOOKS_AND_REFERENCE,4.5,114,Varies with device,"1,000+",Free,0,Mature 17+,Books & Reference,"January 19, 2015",Varies with device,Varies with device


In [4]:
apps = apps.drop(columns=['Type', 'Price', 'Category','Last Updated', 'Current Ver'])
apps.head()

,App,Rating,Reviews,Size,Installs,Content Rating,Genres,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,4.1,159,19M,"10,000+",Everyone,Art & Design,4.0.3 and up
1,Coloring book moana,3.9,967,14M,"500,000+",Everyone,Art & Design;Pretend Play,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",4.7,87510,8.7M,"5,000,000+",Everyone,Art & Design,4.0.3 and up
3,Sketch - Draw & Paint,4.5,215644,25M,"50,000,000+",Teen,Art & Design,4.2 and up
4,Pixel Draw - Number Art Coloring Book,4.3,967,2.8M,"100,000+",Everyone,Art & Design;Creativity,4.4 and up


In [5]:
apps['Android Ver'] = apps['Android Ver'].str.replace('[^\d.]', '')
avgVer = pd.to_numeric(apps['Android Ver'], errors='coerce').mean()
apps['Android Ver'] = apps['Android Ver'].replace('', avgVer)
apps['Android Ver'] = apps['Android Ver'].str.replace('.','a', regex=True).str.replace('a','.', 1, regex=True).str.replace('a','', regex=True)
apps['Size'] = apps['Size'].str.replace('[^\d.]', '')
apps['Installs'] = apps['Installs'].str.replace('[^\d.]', '')

avgSize = pd.to_numeric(apps['Size'], errors='coerce').mean()
apps['Size'] = apps['Size'].replace('', avgSize)
apps = apps.dropna()
apps = apps.astype({'Reviews':'float', 'Size':'float', 'Installs':'float', 'Android Ver':'float'})

C:\Users\rafal\AppData\Local\Temp\ipykernel_19640\2251694235.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  apps['Android Ver'] = apps['Android Ver'].str.replace('[^\d.]', '')
C:\Users\rafal\AppData\Local\Temp\ipykernel_19640\2251694235.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  apps['Size'] = apps['Size'].str.replace('[^\d.]', '')
C:\Users\rafal\AppData\Local\Temp\ipykernel_19640\2251694235.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  apps['Installs'] = apps['Installs'].str.replace('[^\d.]', '')


In [6]:
class DataClean:
    @staticmethod
    def splitGenres(x):
        genresConnected = x["Genres"].unique().tolist()
        genres = set(())
        for i in genresConnected:
            afterSplit = i.split(";")
            for j in afterSplit:
                if j!='February 11, 2018':
                    genres.add(j)
        for i in list(genres):
            x[i] = [0 for x in range(x.shape[0])]
        for index in x.index:
            values = x.loc[index,'Genres'].split(";")
            for i in values:
                x.loc[index,i] = 1
        return x

In [7]:
apps = DataClean.splitGenres(apps)
apps.head()
apps.drop(columns=['Genres'])

,App,Rating,Reviews,Size,Installs,Content Rating,Android Ver,Social,Shopping,Maps & Navigation,...,Trivia,Music,Board,Food & Drink,Educational,Sports,Beauty,Photography,Casino,Action
0,Photo Editor & Candy Camera & Grid & ScrapBook,4.1,159.0,19.0,10000.0,Everyone,4.03,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Coloring book moana,3.9,967.0,14.0,500000.0,Everyone,4.03,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",4.7,87510.0,8.7,5000000.0,Everyone,4.03,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Sketch - Draw & Paint,4.5,215644.0,25.0,50000000.0,Teen,4.20,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Pixel Draw - Number Art Coloring Book,4.3,967.0,2.8,100000.0,Everyone,4.40,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10832,FR Tides,3.8,1195.0,582.0,100000.0,Everyone,2.10,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10833,Chemin (fr),4.8,44.0,619.0,1000.0,Everyone,2.20,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10834,FR Calculator,4.0,7.0,2.6,500.0,Everyone,4.10,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10836,Sya9a Maroc - FR,4.5,38.0,53.0,5000.0,Everyone,4.10,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
class DataProcessing:
    @staticmethod
    def shuffle(x):
        for i in range(len(x)-1,0,-1):
            j=rn.randint(0,i-1)
            x.iloc[j], x.iloc[i] = x.iloc[i], x.iloc[j]
            
    @staticmethod
    def normalization(x):
        values=x.select_dtypes(exclude="object")
        columnNames=values.columns.tolist()
        for column in columnNames[:-1]:
            data = x.loc[:,column]
            max1=max(data)
            min1=min(data)
            for row in range(0,len(x),1):
                xprim=(x.at[row, column]-min1)/(max1-min1)
                x.at[row, column]=xprim
                
    @staticmethod
    def split(x,k):
        return x[:int(len(x)*k//1)], x[int(len(x)*k//1 +1):]

In [9]:
DataProcessing.shuffle(apps)
apps = apps.reset_index(drop=True)

In [10]:
DataProcessing.normalization(apps)

In [11]:
apps.head()

,App,Rating,Reviews,Size,Installs,Content Rating,Genres,Android Ver,Social,Shopping,...,Trivia,Music,Board,Food & Drink,Educational,Sports,Beauty,Photography,Casino,Action
0,Equestria Girls,0.825,8.744955e-03,0.052367,9.999999e-03,Everyone,Role Playing;Action & Adventure,0.428571,0,0,...,0,0,0,0,0,0,0,0,0,0
1,AW Training,0.900,2.227475e-07,0.016113,9.990000e-07,Everyone,Education,0.432857,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Toronto FC,0.600,3.185289e-06,0.078550,9.999000e-06,Teen,Sports,0.471429,0,0,...,0,0,0,0,0,1,0,0,0,0
3,"Candy Camera - selfie, beauty camera, photo ed...",0.850,7.503703e-02,0.036395,1.000000e-01,Everyone,Photography,0.432857,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Pink Diamond Princess Keyboard Theme,0.675,2.404559e-04,0.005841,9.999990e-04,Everyone,Personalization,0.432857,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
trainingSet, validatingSet = DataProcessing.split(apps, 0.999)
validatingSet = validatingSet.reset_index(drop=True)
validatingSet

,App,Rating,Reviews,Size,Installs,Content Rating,Genres,Android Ver,Social,Shopping,...,Trivia,Music,Board,Food & Drink,Educational,Sports,Beauty,Photography,Casino,Action
0,TRANSFORMERS: Forged to Fight,0.900,0.006503,0.089627,0.01000,Everyone 10+,Action,0.432857,0,0,...,0,0,0,0,0,0,0,0,0,1
1,"Goal Meter: Goal Tracker, Habit Changer,To-Do ...",0.900,0.000177,0.019134,0.00010,Everyone,Productivity,0.485714,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"busuu: Learn Languages - Spanish, English & More",0.825,0.004617,0.020141,0.01000,Everyone 10+,Education,0.571429,0,0,...,0,0,0,0,0,0,0,0,0,0
3,F-Stop Gallery Pro,0.850,0.000029,0.057402,0.00001,Everyone,Photography,0.171429,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Dr. Driving,0.875,0.110755,0.008963,0.10000,Everyone,Racing,0.432857,0,0,...,0,0,0,0,0,0,0,0,0,0
5,White Noise Baby,0.875,0.000235,0.043303,0.00100,Everyone,Lifestyle,0.442857,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Dresses Ideas & Fashions +3000,0.875,0.000011,0.007251,0.00010,Mature 17+,Beauty,0.085714,0,0,...,0,0,0,0,0,0,1,0,0,0
7,CBS Sports Fantasy,0.750,0.000971,0.026183,0.00100,Everyone,Sports,0.571429,0,0,...,0,0,0,0,0,1,0,0,0,0


In [13]:
apps["Content Rating"].unique().tolist()

['Everyone',
 'Teen',
 'Everyone 10+',
 'Mature 17+',
 'Adults only 18+',
 'Unrated']

In [14]:
if 'Mature 17+' in validatingSet["Content Rating"].unique().tolist() or 'Adults only 18+' in validatingSet["Content Rating"].unique().tolist():
    validatingSet["Content Rating"] = ['Adults only 18+' for i in range(len(validatingSet))]
elif 'Teen' in validatingSet["Content Rating"].unique().tolist():
    validatingSet["Content Rating"] = ['Teen' for i in range(len(validatingSet))]

In [15]:
validatingSet.head()

,App,Rating,Reviews,Size,Installs,Content Rating,Genres,Android Ver,Social,Shopping,...,Trivia,Music,Board,Food & Drink,Educational,Sports,Beauty,Photography,Casino,Action
0,TRANSFORMERS: Forged to Fight,0.900,0.006503,0.089627,0.01000,Adults only 18+,Action,0.432857,0,0,...,0,0,0,0,0,0,0,0,0,1
1,"Goal Meter: Goal Tracker, Habit Changer,To-Do ...",0.900,0.000177,0.019134,0.00010,Adults only 18+,Productivity,0.485714,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"busuu: Learn Languages - Spanish, English & More",0.825,0.004617,0.020141,0.01000,Adults only 18+,Education,0.571429,0,0,...,0,0,0,0,0,0,0,0,0,0
3,F-Stop Gallery Pro,0.850,0.000029,0.057402,0.00001,Adults only 18+,Photography,0.171429,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Dr. Driving,0.875,0.110755,0.008963,0.10000,Adults only 18+,Racing,0.432857,0,0,...,0,0,0,0,0,0,0,0,0,0


<h2>k-nearest neighbors algorithm</h2>

In [16]:
class KNN:
    @staticmethod
    def distance(v1, v2):
        sum = 0
        for j in range(1,len(v1)):
            if j != 5 and j!=6 and j!=7:
                sum += (v1[j] - v2[j])**2
        return sum**(1/2)
    def clustering(dataset, sample):
        distances = []
        for i in range(len(dataset)):
            distances.append(KNN.distance(sample, dataset.iloc[i]))
            if dataset.iloc[i]['Android Ver'] > sample['Android Ver']:
                distances[i] = 10
            if sample['Content Rating'] != 'Adults only 18+':
                if dataset.iloc[i]['Content Rating'] in ['Mature 17+','Adults only 18+']:
                    distances[i] = 10
            elif sample['Content Rating'] != 'Teen':
                if dataset.iloc[i]['Content Rating'] in ['Teen','Mature 17+','Adults only 18+']:
                    distances[i] = 10
        dataset["distance"] = distances
                    
        return dataset
    
    def recommendation(dataset,samples):
        sorted = []
        dataset["distance"] = [10 for x in range(len(dataset))]
        for index in range(len(samples)):
            sorted.append(KNN.clustering(dataset.copy(),samples.iloc[index]))
        distances = []
        for index in dataset.index:
#             dataset.loc[index,'distance']
            distance = []
            for i in range(len(sorted)):
                distance.append(sorted[i].loc[index,'distance'])
            avg = []
            for i in range(3):
                avg.append(min(distance))
                distance.remove(min(distance))
            distances.append(sum(avg)/3)
        dataset["distance"] = distances
        dataset = dataset.sort_values("distance")
        return dataset.head(10)

<h2>Apps Recommendation</h2>

<h4>Method 1</h4>

In [17]:
avgRating = pd.to_numeric(validatingSet['Rating'], errors='coerce').mean()
avgReviews = pd.to_numeric(validatingSet['Reviews'], errors='coerce').mean()
avgSize = pd.to_numeric(validatingSet['Size'], errors='coerce').mean()
avgInstalls = pd.to_numeric(validatingSet['Installs'], errors='coerce').mean()
maxAndroidVer = max(validatingSet['Android Ver'])
genres = []
for i in validatingSet.columns.tolist()[8:]:
    genres.append(pd.to_numeric(validatingSet[i], errors='coerce').mean())
avgAppList = ['avgApp',avgRating,avgReviews,avgSize,avgInstalls,validatingSet.iloc[0]['Content Rating'],"avgGenre",maxAndroidVer]
for i in genres:
    avgAppList.append(i)
avgApp = pd.DataFrame(columns=validatingSet.columns.tolist())
avgApp.loc[0] = avgAppList
recommendedApps = KNN.clustering(trainingSet.copy(), validatingSet.iloc[0])
recommendedApps.sort_values("distance").head()

,App,Rating,Reviews,Size,Installs,Content Rating,Genres,Android Ver,Social,Shopping,...,Music,Board,Food & Drink,Educational,Sports,Beauty,Photography,Casino,Action,distance
174,Alto's Adventure,0.900,0.011477,0.062437,0.01,Everyone,Action,0.428571,0,0,...,0,0,0,0,0,0,0,0,1,0.027642
4750,Alto's Adventure,0.900,0.011486,0.062437,0.01,Everyone,Action,0.428571,0,0,...,0,0,0,0,0,0,0,0,1,0.027643
4847,Dude Perfect 2,0.875,0.008942,0.069486,0.01,Everyone,Action,0.432857,0,0,...,0,0,0,0,0,0,0,0,1,0.032196
5980,Dude Perfect 2,0.875,0.008947,0.069486,0.01,Everyone,Action,0.432857,0,0,...,0,0,0,0,0,0,0,0,1,0.032197
6293,Bomber Friends,0.875,0.017301,0.048338,0.01,Everyone 10+,Action,0.432857,0,0,...,0,0,0,0,0,0,0,0,1,0.049461


In [18]:
validatingSet.head()

,App,Rating,Reviews,Size,Installs,Content Rating,Genres,Android Ver,Social,Shopping,...,Trivia,Music,Board,Food & Drink,Educational,Sports,Beauty,Photography,Casino,Action
0,TRANSFORMERS: Forged to Fight,0.900,0.006503,0.089627,0.01000,Adults only 18+,Action,0.432857,0,0,...,0,0,0,0,0,0,0,0,0,1
1,"Goal Meter: Goal Tracker, Habit Changer,To-Do ...",0.900,0.000177,0.019134,0.00010,Adults only 18+,Productivity,0.485714,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"busuu: Learn Languages - Spanish, English & More",0.825,0.004617,0.020141,0.01000,Adults only 18+,Education,0.571429,0,0,...,0,0,0,0,0,0,0,0,0,0
3,F-Stop Gallery Pro,0.850,0.000029,0.057402,0.00001,Adults only 18+,Photography,0.171429,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Dr. Driving,0.875,0.110755,0.008963,0.10000,Adults only 18+,Racing,0.432857,0,0,...,0,0,0,0,0,0,0,0,0,0


<h4>Method 2</h4>

In [19]:
KNN.recommendation(trainingSet.copy(),validatingSet.copy())

,App,Rating,Reviews,Size,Installs,Content Rating,Genres,Android Ver,Social,Shopping,...,Music,Board,Food & Drink,Educational,Sports,Beauty,Photography,Casino,Action,distance
3591,Babbel – Learn Languages,0.825,5.945887e-03,0.020141,9.999999e-03,Everyone,Education,0.485714,0,0,...,0,0,0,0,0,0,0,0,0,0.944609
3731,Babbel – Learn Languages,0.825,5.946177e-03,0.020141,9.999999e-03,Everyone,Education,0.485714,0,0,...,0,0,0,0,0,0,0,0,0,0.944609
2566,Babbel – Learn Languages,0.825,5.946177e-03,0.020141,9.999999e-03,Everyone,Education,0.485714,0,0,...,0,0,0,0,0,0,0,0,0,0.944609
3269,Babbel – Learn Languages,0.825,5.964866e-03,0.020141,9.999999e-03,Everyone,Education,0.485714,0,0,...,0,0,0,0,0,0,0,0,0,0.944615
6982,New Holland Agriculture T5 EC,0.900,2.450222e-07,0.014099,9.990000e-07,Everyone,Productivity,0.432857,0,0,...,0,0,0,0,0,0,0,0,0,0.945344
357,Speed Boat Racing,0.750,5.537502e-05,0.026183,9.999990e-04,Everyone,Sports,0.432857,0,0,...,0,0,0,0,1,0,0,0,0,0.945665
2314,Hovercraft Racer,0.750,4.833620e-06,0.027190,9.999900e-05,Everyone,Sports,0.442857,0,0,...,0,0,0,0,1,0,0,0,0,0.945913
5985,Business Calendar 2,0.900,2.285233e-03,0.014099,4.999999e-03,Everyone,Productivity,0.442857,0,0,...,0,0,0,0,0,0,0,0,0,0.946101
4941,Inventory & Barcode scanner & WIFI scanner,0.900,4.185425e-05,0.017120,4.999900e-05,Everyone,Productivity,0.471429,0,0,...,0,0,0,0,0,0,0,0,0,0.946814
2070,My Study Life - School Planner,0.825,1.065758e-03,0.020141,9.999990e-04,Everyone,Education,0.432857,0,0,...,0,0,0,0,0,0,0,0,0,0.947055


<h2>Importing Reviews Database</h2>

In [20]:
reviews = pd.read_csv(r"processedReviews.csv")

In [21]:
display(reviews)

,App,reviews
0,10 Best Foods for You,good;best;great;food;healthy
1,104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,easy;good;job;nice;mobile
2,11st,and;not;you;for;there
3,1800 Contacts - Lens Store,easy;order;super;love;contacts
4,1LINE – One Line with One Touch,game;love;nice;many;ads
...,...,...
860,Hotels.com: Book Hotel Rooms & Find Vacation D...,hotel;book;pet;find;good
861,Hotspot Shield Free VPN Proxy & Wi-Fi Security,good;7;days;free;trial
862,Hotstar,shows;answer;u;please;time
863,Hotwire Hotel & Car Rental App,would;hotels;hotel;app.;they


In [22]:
appsWithReviews = pd.merge(apps, reviews, on="App")

In [23]:
display(appsWithReviews)

,App,Rating,Reviews,Size,Installs,Content Rating,Genres,Android Ver,Health & Fitness,Dating,...,Shopping,Communication,Parenting,Casual,Music & Audio,Entertainment,Puzzle,Creativity,Adventure,reviews
0,Cookpad,0.875,0.002931,0.007251,0.01000,Everyone,Food & Drink,0.457143,0,0,...,0,0,0,0,0,0,0,0,0,recipes;search;helpful;love;cooking
1,Golf GPS by SwingxSwing,0.900,0.000828,0.036395,0.00100,Everyone,Sports,0.485714,0,0,...,0,0,0,0,0,0,0,0,0,update;round;app.;score;using
2,Candy Day,0.875,0.000261,0.047331,0.00100,Everyone,Casual;Brain Games,0.428571,0,0,...,0,0,0,1,0,0,0,0,0,game;love;play;fun;really
3,Family Dollar,0.825,0.000058,0.020141,0.00100,Everyone,Shopping,0.442857,0,0,...,1,0,0,0,0,0,0,0,0,dollar;family;store;love;time
4,HTC Sense Input-AR,0.700,0.000032,0.001712,0.00010,Everyone,Tools,0.442857,0,0,...,0,0,0,0,0,0,0,0,0,phone;good;htc;need;nice
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899,Cooking Madness - A Chef's Restaurant Games,0.925,0.007993,0.048338,0.01000,Everyone,Arcade,0.442857,0,0,...,0,0,0,0,0,0,0,0,0,game;get;game.;really;make
900,HD Video Player,0.825,0.000035,0.001913,0.00100,Everyone,Video Players & Editors,0.442857,0,0,...,0,0,0,0,0,0,0,0,0,good;player;video;great;videos
901,Herpes Positive Singles Dating,0.850,0.000004,0.027190,0.00001,Mature 17+,Dating,0.442857,0,1,...,0,0,0,0,0,0,0,0,0,dating;people;great;app.;herpes
902,Anime Manga Coloring Book,0.875,0.000112,0.010070,0.00010,Everyone,Art & Design,0.428571,0,0,...,0,0,0,0,0,0,0,0,0,love;color;good;anime;thing


In [25]:
trainingSet, validatingSet = DataProcessing.split(appsWithReviews, 0.995)
validatingSet = validatingSet.reset_index(drop=True)
validatingSet

,App,Rating,Reviews,Size,Installs,Content Rating,Genres,Android Ver,Health & Fitness,Dating,...,Shopping,Communication,Parenting,Casual,Music & Audio,Entertainment,Puzzle,Creativity,Adventure,reviews
0,HD Video Player,0.825,0.000035,0.001913,0.00100,Everyone,Video Players & Editors,0.442857,0,0,...,0,0,0,0,0,0,0,0,0,good;player;video;great;videos
1,Herpes Positive Singles Dating,0.850,0.000004,0.027190,0.00001,Mature 17+,Dating,0.442857,0,1,...,0,0,0,0,0,0,0,0,0,dating;people;great;app.;herpes
2,Anime Manga Coloring Book,0.875,0.000112,0.010070,0.00010,Everyone,Art & Design,0.428571,0,0,...,0,0,0,0,0,0,0,0,0,love;color;good;anime;thing
3,Harkins Theatres,0.825,0.000034,0.011078,0.00010,Everyone,Entertainment,0.442857,0,0,...,0,0,0,0,0,1,0,0,0,app.;get;love;hate;harkins


In [26]:
if 'Mature 17+' in validatingSet["Content Rating"].unique().tolist() or 'Adults only 18+' in validatingSet["Content Rating"].unique().tolist():
    validatingSet["Content Rating"] = ['Adults only 18+' for i in range(len(validatingSet))]
elif 'Teen' in validatingSet["Content Rating"].unique().tolist():
    validatingSet["Content Rating"] = ['Teen' for i in range(len(validatingSet))]

<h3>Method 1</h3>

In [27]:
class KNNWithJaccardSimilarityCoefficient:
    @staticmethod
    def distance(v1, v2):
        sum = 0
        for j in range(1,len(v1)-1):
            if j != 5 and j!=6 and j!=7:
                sum += (v1[j] - v2[j])**2
        return (sum**(1/2)) * KNNWithJaccardSimilarityCoefficient.jaccard(v1[len(v1)-1], v2[len(v1)-1])
    
    def jaccard(string1,string2):
        sim = 0
        string1 = string1.split(";")
        string2 = string2.split(";")
        for word in string1:
            if word in string2:
                sim+=1
        return 1 - (sim/5)
    
    def clustering(dataset, sample):
        distances = []
        for i in range(len(dataset)):
            distances.append(KNNWithJaccardSimilarityCoefficient.distance(sample, dataset.iloc[i]))
            if dataset.iloc[i]['Android Ver'] > sample['Android Ver']:
                distances[i] = 10
            if sample['Content Rating'] != 'Adults only 18+':
                if dataset.iloc[i]['Content Rating'] in ['Mature 17+','Adults only 18+']:
                    distances[i] = 10
            elif sample['Content Rating'] != 'Teen':
                if dataset.iloc[i]['Content Rating'] in ['Teen','Mature 17+','Adults only 18+']:
                    distances[i] = 10
        dataset["distance"] = distances
                    
        return dataset
    
    def recommendation(dataset,samples):
        sorted = []
        dataset["distance"] = [10 for x in range(len(dataset))]
        for index in range(len(samples)):
            sorted.append(KNNWithJaccardSimilarityCoefficient.clustering(dataset.copy(),samples.iloc[index]))
        distances = []
        for index in dataset.index:
#             dataset.loc[index,'distance']
            distance = []
            for i in range(len(sorted)):
                distance.append(sorted[i].loc[index,'distance'])
            avg = []
            for i in range(3):
                avg.append(min(distance))
                distance.remove(min(distance))
            distances.append(sum(avg)/3)
        dataset["distance"] = distances
        dataset = dataset.sort_values("distance")
        return dataset.head(10)

In [28]:
KNNWithJaccardSimilarityCoefficient.recommendation(trainingSet.copy(),validatingSet.copy())

,App,Rating,Reviews,Size,Installs,Content Rating,Genres,Android Ver,Health & Fitness,Dating,...,Communication,Parenting,Casual,Music & Audio,Entertainment,Puzzle,Creativity,Adventure,reviews,distance
869,H TV,0.825,0.002296,0.004632,0.00500,Everyone,Entertainment,0.190000,0,0,...,0,0,0,0,1,0,0,0,good;tv;many;channel;channels,0.757293
870,H TV,0.825,0.002296,0.004632,0.00500,Everyone,Entertainment,0.190000,0,0,...,0,0,0,0,1,0,0,0,good;tv;many;channel;channels,0.757293
811,Garden Coloring Book,0.850,0.000307,0.032226,0.00100,Everyone,Art & Design,0.285714,0,0,...,0,0,0,0,0,0,0,0,love;game;coloring;color;good,0.758950
837,Eat Fit - Diet and Health Free,0.775,0.000055,0.002618,0.00010,Everyone,Health & Fitness,0.185714,1,0,...,0,0,0,0,0,0,0,0,great;really;good;get;love,0.849597
360,Color by Number - Draw Sandbox Pixel Art,0.900,0.000228,0.014099,0.00100,Everyone,Entertainment,0.442857,0,0,...,0,0,0,0,1,0,0,0,game;good;fun;pictures;love,0.869405
888,Floor Plan Creator,0.775,0.000816,0.036395,0.00500,Everyone,Art & Design,0.190000,0,0,...,0,0,0,0,0,0,0,0,add;could;3d;good;plan,0.876857
468,A Call From Santa Claus!,0.725,0.000083,0.010070,0.00100,Everyone,Entertainment,0.432857,0,0,...,0,0,0,0,1,0,0,0,santa;call;good;real;said,0.884685
213,High Blood Pressure Symptoms,0.825,0.000001,0.001007,0.00001,Everyone,Medical,0.432857,0,0,...,0,0,0,0,0,0,0,0,good;great;app.;medical;pay,0.942911
375,Banjo,0.800,0.001300,0.008359,0.00100,Everyone,Social,0.185714,0,0,...,0,0,0,0,0,0,0,0,news;great;get;account;app.,0.943119
212,High Blood Pressure Symptoms,0.750,0.000012,0.001309,0.00010,Everyone,Medical,0.285714,0,0,...,0,0,0,0,0,0,0,0,good;great;app.;medical;pay,0.944499
